In [ ]:
import os
from random import shuffle
import json

import tensorflow as tf
import matplotlib.pyplot as plt

from src.models.monuseg.light_models import get_model
from src.data.monuseg.tf_data import get_dataset, tf_random_crop
from src.models.loss import dice_coe_loss, dice_coe_metric

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

path_indices = "/home/valentin/python_wkspce/2d_bispectrum_cnn/data/indices/monuseg.json"
split = 0

In [ ]:
w_fg = 1.9
w_border = 5.0
w_bg = 0.44
def loss(y_true, y_pred):
    l = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    w = (y_true[..., 0] * w_fg + y_true[..., 1] * w_border +
         y_true[..., 2] * w_bg)
    return tf.reduce_mean(w * l, axis=(1, 2))

In [ ]:
with open(path_indices, "r") as f:
    indices_list = json.load(f)

ds_train = get_dataset(id_list=indices_list[split]["train"]).take(1)
ds_train = ds_train.cache().repeat(15)
ds_train = ds_train.map(lambda image, seg: tf_random_crop(
    image, seg, rotation_angle="right-angle")).batch(4)

ds_val = get_dataset(id_list=indices_list[split]["val"])
ds_val = ds_val.cache().batch(1)

ds_val_instance = get_dataset(id_list=indices_list[split]["val"],
                              instance=True)
ds_val_instance = ds_val_instance.cache().batch(1)

ds_test = get_dataset(id_list=indices_list[split]["test"], instance=True)
ds_test = ds_test.cache().batch(1)


In [ ]:
model = get_model(
    model_name="BispectUnetLight",
    output_channels=3,
    n_harmonics=3,
    n_feature_maps=[12, 24, 48],
    radial_profile_type="disks",
)


In [ ]:
x, y= next(ds_train.as_numpy_iterator())

In [ ]:
y_pred = model(x)

In [ ]:
model.summary()

In [ ]:
unet = get_model(
    model_name="UnetLight",
    output_channels=3,
    n_feature_maps=[8, 16, 32],
)


In [ ]:
y_pred = unet(x)

In [ ]:
unet.summary()